In [1]:
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, train_test_split, LeaveOneOut
from tqdm.autonotebook import tqdm
from sklearn import preprocessing
from sklearn.metrics import classification_report, cohen_kappa_score
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA


import sys

import matplotlib
import seaborn as sns
import scipy.io as sio
import scipy as sc
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
# %matplotlib outline
sns.set()

plt.style.use('seaborn-white')

sys.path.append("../src/data/")
sys.path.append("../src/tools/")
sys.path.append("../src/models/")
sys.path.append("../src/visualization/")

/home/brain/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
from file import read_file, get_bad_trials, get_bad_trials_comportamental, get_time_delay
from conversion import to_DataFrame, split_exposure, merge_and_clean, merge_export, to_DataFrame_autoenconder
from exposure import _exposure_1, _exposure_2, _exposure_1_bad, _exposure_2_bad, get_group_time, fixing_bad_trials, get_last_125ms


1 items had no tests:
    __main__
0 tests in 1 items.
0 passed and 0 failed.
Test passed.


In [3]:
import tensorflow as tf

/home/brain/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/brain/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/brain/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/brain/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [8]:
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [9]:
N_PEOPLE = 20
PATH_AUD = '../data/raw/aud'
PATH_VIS = '../data/raw/vis'
PATH_INFO = '../data/raw/info_'

N_CHANNELS = 64
#N_CHANNELS = 62

ORIGINAL_FREQUENCY = 1000

DOWN_SAMPLED_FREQUENCY = 250

In [10]:
data_aud, data_vis, CHANNEL_NAMES = read_file(PATH_AUD, PATH_VIS)

In [11]:
## Splitting the exposures for all the individuals

aud_1 = _exposure_1(data_aud)

aud_2 = _exposure_2(data_aud) 

vis_1 = _exposure_1(data_vis)

vis_2 = _exposure_2(data_vis)

In [12]:
bad_trials_aud, bad_trials_vis = get_bad_trials(PATH_AUD, PATH_VIS)

In [13]:
## Splitting the exposures for all the individuals

bad_aud_1 = _exposure_1_bad(bad_trials_aud, 'Aud')

bad_aud_2 = _exposure_2_bad(bad_trials_aud, 'Aud') 

bad_vis_1 = _exposure_1_bad(bad_trials_vis, 'Vis')

bad_vis_2 = _exposure_2_bad(bad_trials_vis, 'Vis')

In [14]:
bad_vis_comport = get_bad_trials_comportamental('vis')
bad_aud_comport = get_bad_trials_comportamental('aud')

In [15]:
clean_aud_1, clean_aud_2, clean_vis_1, clean_vis_2 = fixing_bad_trials(
    bad_aud_1, bad_aud_2, bad_vis_1, bad_vis_2)

In [16]:
clean_aud_1 = clean_aud_1.append(bad_aud_comport,ignore_index=True)
clean_aud_2 = clean_aud_2.append(bad_aud_comport,ignore_index=True)

clean_vis_1 = clean_vis_1.append(bad_vis_comport,ignore_index=True)
clean_vis_2 = clean_vis_2.append(bad_vis_comport,ignore_index=True)

In [17]:
indice_s2_vis = get_time_delay('vis')
indice_s2_aud = get_time_delay('aud')

time_s2_vis = get_time_delay('vis', export_as_indice=False)
time_s2_aud = get_time_delay('aud', export_as_indice=False)

In [18]:
aud_1_average = list(map(get_last_125ms, aud_1, indice_s2_aud))
aud_2_average = list(map(get_last_125ms, aud_2, indice_s2_aud))


vis_1_average = list(map(get_last_125ms, vis_1, indice_s2_vis))
vis_2_average = list(map(get_last_125ms, vis_2, indice_s2_vis))

In [19]:
classes_aud, classes_vis = get_group_time(time_s2_aud, time_s2_vis)

In [20]:
#Getting in format of dataframe:

df_aud_1_aver = to_DataFrame(aud_1_average,classes_aud,CHANNEL_NAMES)
df_aud_2_aver = to_DataFrame(aud_2_average,classes_aud,CHANNEL_NAMES)

df_vis_1_aver = to_DataFrame(vis_1_average,classes_vis,CHANNEL_NAMES)
df_vis_2_aver = to_DataFrame(vis_2_average,classes_vis,CHANNEL_NAMES)

In [22]:
import torch
from torch import nn, optim


In [24]:
gpu=1
device = torch.device(f"cuda:{gpu}" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(device)


RuntimeError: cuda runtime error (101) : invalid device ordinal at /pytorch/torch/csrc/cuda/Module.cpp:59

In [21]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [35]:
def eval_mse(y_pred, y_true):
    """
    Evaluates mean square error (MSE) between y_pred and y_true

    Args:
      y_pred (torch.Tensor)
          prediction samples

      v (numpy array of floats)
          ground truth samples

    Returns:
      MSE(y_pred, y_true)
    """

    with torch.no_grad():
        criterion = nn.MSELoss()
        loss = criterion(y_pred, y_true)

    return float(loss)


def eval_bce(y_pred, y_true):
    """
    Evaluates binary cross-entropy (BCE) between y_pred and y_true

    Args:
      y_pred (torch.Tensor)
          prediction samples

      v (numpy array of floats)
          ground truth samples

    Returns:
      BCE(y_pred, y_true)
    """

    with torch.no_grad():
        criterion = nn.BCELoss()
        loss = criterion(y_pred, y_true)

    return float(loss)


def runSGD(net, input_train, input_test, criterion='bce',
           n_epochs=10, batch_size=32, verbose=False):
    """
    Trains autoencoder network with stochastic gradient descent with Adam
    optimizer and loss criterion. Train samples are shuffled, and loss is
    displayed at the end of each opoch for both MSE and BCE. Plots training loss
    at each minibatch (maximum of 500 randomly selected values).

    Args:
      net (torch network)
          ANN object (nn.Module)

      input_train (torch.Tensor)
          vectorized input images from train set

      input_test (torch.Tensor)
          vectorized input images from test set

      criterion (string)
          train loss: 'bce' or 'mse'

      n_epochs (boolean)
          number of full iterations of training data

      batch_size (integer)
          number of element in mini-batches

      verbose (boolean)
          print final loss

    Returns:
      Nothing.
    """

    # Initialize loss function
    if criterion == 'mse':
        loss_fn = nn.MSELoss()
    elif criterionpytotch == 'bce':
        loss_fn = nn.BCELoss()
    else:
        print('Please specify either "mse" or "bce" for loss criterion')

    # Initialize SGD optimizer
    optimizer = optim.Adam(net.parameters())

    # Placeholder for loss
    track_loss = []

    # print('Epoch', '\t', 'Loss train', '\t', 'Loss test')
    for i in range(n_epochs):

        shuffle_idx = np.random.permutation(len(input_train))
        batches = torch.split(input_train[shuffle_idx], batch_size)

        for batch in batches:

            output_train = net(batch).to(device)
            loss = loss_fn(output_train, batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Keep track of loss at each epoch
            track_loss += [float(loss)]

        loss_epoch = f'{i+1}/{n_epochs}'
        with torch.no_grad():
            output_train = net(input_train).cuda()
            loss_train = loss_fn(output_train, input_train)
            loss_epoch += f'\t {loss_train:.4f}'
            output_test = net(input_test.cuda()).to(device)
            loss_test = loss_fn(output_test, input_test)
            loss_epoch += f'\t\t {loss_test:.4f}'

        # print(loss_epoch)    
    
    if verbose:
        # Print loss
        loss_mse = f'\nMSE\t {eval_mse(output_train, input_train):0.4f}'
        loss_mse += f'\t\t {eval_mse(output_test, input_test):0.4f}'
        print(loss_mse)

        loss_bce = f'BCE\t {eval_bce(output_train, input_train):0.4f}'
        loss_bce += f'\t\t {eval_bce(output_test, input_test):0.4f}'
        print(loss_bce)

        # Plot loss
        step = int(np.ceil(len(track_loss) / 500))
        x_range = np.arange(0, len(track_loss), step)
        plt.figure()
        plt.plot(x_range, track_loss[::step], 'C0')
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.xlim([0, None])
        plt.ylim([0, None])
        plt.show()

    return net

class NormalizeLayer(nn.Module):
    """
    pyTorch layer (nn.Module) that normalizes activations by their L2 norm.

    Args:
        None.

    Returns:
        Object inherited from nn.Module class.
    """

    def __init__(self):
        super().__init__()

    def forward(self, x):
        return nn.functional.normalize(x, p=2, dim=1)


def init_weights_kaiming_normal(layer):
    """
    Initializes weights from linear PyTorch layer
    with kaiming normal distribution.

    Args:
      layer (torch.Module)
          Pytorch layer

    Returns:nossa
      Nothing.
    """
    # check for linear PyTorch layer
    if isinstance(layer, nn.Linear):
        # initialize weights with kaiming normal distribution
        nn.init.kaiming_normal_(layer.weight.data,dtype=)


def init_weights_kaiming_uniform(layer):
    """
    Initializes weights from linear PyTorch layer
    with kaiming uniform distribution.

    Args:
      layer (torch.Module)
          Pytorch layer

    Returns:
      Nothing.
    """
    # check for linear PyTorch layer
    if isinstance(layer, nn.Linear):
        # initialize weights with kaiming uniform distribution
        nn.init.kaiming_uniform_(layer.weight.data)

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [34]:
encoding_size = 2#[2,4,8,16]
input_size = 62
model_32 = nn.Sequential(
    nn.Linear(input_size, int(input_size / 2)),         #32
    # Add activation function
    nn.PReLU(),
    # Add another layer
    nn.Linear(int(input_size / 2), int(input_size / 4)),#16
    # Add activation function
    nn.PReLU(),
    # Add another layer
    nn.Linear(int(input_size / 4), int(input_size / 8)),#8
    # Add activation function
    nn.PReLU(),
    # Add another layer
    nn.Linear(int(input_size / 8), encoding_size),      #4
    # Add activation function
    nn.PReLU(),
    # Add another layer
    nn.Linear(encoding_size, int(input_size / 8)),     #2
    # Add activation function
    nn.PReLU(),
    # Add another layer
    nn.Linear(int(input_size / 8), int(input_size / 4)),#8
    # Add activation function
    nn.PReLU(), 
    # Add another layer
    nn.Linear(int(input_size / 4), int(input_size / 2)),#16
    # Add activation function
    nn.PReLU(),
    # Add another layeré´
    nn.Linear(int(input_size / 2), input_size),#32    
    # Add another layer
    nn.Sigmoid()
    )
match_case=False
model_32[:-2].apply(init_weights_kaiming_normal)

print(f'Autoencoder \n\n {model_32}\n')


model_32 = model_32.to(device)

torch.backends.cudnn.benchmark=True

Autoencoder 

 Sequential(
  (0): Linear(in_features=62, out_features=31, bias=True)
  (1): PReLU(num_parameters=1)
  (2): Linear(in_features=31, out_features=15, bias=True)
  (3): PReLU(num_parameters=1)
  (4): Linear(in_features=15, out_features=7, bias=True)
  (5): PReLU(num_parameters=1)
  (6): Linear(in_features=7, out_features=2, bias=True)
  (7): PReLU(num_parameters=1)
  (8): Linear(in_features=2, out_features=7, bias=True)
  (9): PReLU(num_parameters=1)
  (10): Linear(in_features=7, out_features=15, bias=True)
  (11): PReLU(num_parameters=1)
  (12): Linear(in_features=15, out_features=31, bias=True)
  (13): PReLU(num_parameters=1)
  (14): Linear(in_features=31, out_features=62, bias=True)
  (15): Sigmoid()
)



In [21]:
from my_autoenconder import AutoEnconder

In [46]:
def classification_within_modality_auto(dataFrame, categoria, exposure):
    '''






    '''
    dataFrame_result = []
    loo = LeaveOneOut()

    pbar = tqdm(total=loo.get_n_splits(dataFrame))

    for ind, pearson in dataFrame.groupby('people'):

        X = pearson.drop(['trial', 'group', 'people'], 1)
        y = pearson['group']

        loo = LeaveOneOut()

        for train_index, test_index in loo.split(X):

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Normalize
            train_mean = average(X_train, axis=0)

            X_train_without_mean = subtract(X_train, train_mean)
            X_test_without_mean = subtract(X_test, train_mean)

            clf = GaussianNB()

            clf.class_prior_ = [(1/6), (1/6), (1/6), (1/6), (1/6), (1/6)]

            # print(X_train_without_mean.to_numpy().shape)

            my_autoenconder
            
            ae = runSGD(model_32, X_train_torch,
                        X_test_torch,
                        criterion='mse', n_epochs=100, batch_size=16, verbose=False)

            encoder = ae[:7]
            
            with torch.no_grad():

                X_train_pca = encoder(X_train_torch)
                X_test_pca = encoder(X_test_torch)

            clf = clf.fit(X_train_pca, y_train)

            y_pred = clf.predict(X_test_pca)

            dataFrame_result.append(
                [ind, y_pred, y_test.values, categoria, exposure])
            
            #print(dataFrame_result)
            pbar.update(1)

    return dataFrame_result

In [18]:
from sklearn.model_selection import LeaveOneOut
from tqdm.autonotebook import tqdm
from numpy import average, subtract
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from pandas import DataFrame, concat

In [19]:
torch.cuda.is_available() 

True

In [48]:
classification_within_modality_auto(df_aud_1_aver, 'Auditory','E1')

RuntimeError: CUDA error: an illegal memory access was encountered

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

NameError: name 'tensor' is not defined